In [18]:
from IPython.display import display, Markdown  # 在jupyter显示信息的工具

from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
# from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

import pandas as pd

In [17]:
# 加载中文数据
file = "../data/product_data.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

# 查看数据
test_data = pd.read_csv(file, skiprows=0)
display(test_data.shape)
display(test_data.head())

(25, 2)

,product_name,description
0,全自动咖啡机,规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' ...
1,电动牙刷,规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的...
2,橙味维生素C泡腾片,规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维...
3,无线蓝牙耳机,规格:\n单个耳机尺寸：1.5'' x 1.3''。\n\n为什么我们热爱它:\n这款无线蓝...
4,瑜伽垫,规格:\n尺寸：24'' x 68''。\n\n为什么我们热爱它:\n我们的瑜伽垫拥有出色的...


In [13]:
# 创建 Ollama 嵌入模型实例
embeddings = OllamaEmbeddings(model="qwen2.5")  # 或其他在 Ollama 中安装的模型名称
# 将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch, embedding=embeddings,  # 添加 embedding 参数
).from_loaders(
    [loader]
)

# 通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"},
)

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [22]:
data[10]

Document(metadata={'source': '../data/product_data.csv', 'row': 10}, page_content="product_name: 高清电视机\ndescription: 规格:\n尺寸：50''。\n\n为什么我们热爱它:\n我们的高清电视机拥有出色的画质和强大的音效，带来沉浸式的观看体验。\n\n材质与护理:\n使用干布清洁。\n\n构造:\n由塑料、金属和电子元件制成。\n\n其他特性:\n支持网络连接，可以在线观看视频。\n配备遥控器。\n在韩国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

In [21]:
data[11]

Document(metadata={'source': '../data/product_data.csv', 'row': 11}, page_content="product_name: 旅行背包\ndescription: 规格:\n尺寸：18'' x 12'' x 6''。\n\n为什么我们热爱它:\n我们的旅行背包拥有多个实用的内外袋，轻松装下您的必需品，是短途旅行的理想选择。\n\n材质与护理:\n可以手洗，自然晾干。\n\n构造:\n由防水尼龙制成。\n\n其他特性:\n附带可调节背带和安全锁。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

In [25]:
examples = [
    {"query": "高清电视机怎么进行护理？", "answer": "使用干布清洁。"},
    {"query": "旅行背包有内外袋吗？", "answer": "有。"},
]
print(type(examples))
print(examples[0])

<class 'list'>
{'query': '高清电视机怎么进行护理？', 'answer': '使用干布清洁。'}


In [ ]:
from langchain.evaluation.qa import (
    QAGenerateChain,
)  # 导入QA生成链，它将接收文档，并从每个文档中创建一个问题答案对

# 下面是langchain.evaluation.qa.generate_prompt中的源码，在template的最后加上“请使用中文输出”
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import PromptTemplate
from langchain.base_language import BaseLanguageModel
from typing import Any

template = """You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>
请使用中文输出
"""
output_parser = RegexParser(
    regex=r"QUESTION: (.*?)\nANSWER: (.*)", output_keys=["query", "answer"]
)
PROMPT = PromptTemplate(
    input_variables=["doc"], template=template, output_parser=output_parser
)


# 继承QAGenerateChain
class ChineseQAGenerateChain(QAGenerateChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT, **kwargs)


example_gen_chain = ChineseQAGenerateChain.from_llm(
    ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型
)  # 通过传递 ollama 语言模型来创建这个链
new_examples = example_gen_chain.apply([{"doc": t} for t in data[:5]])

# 查看用例数据
new_examples

[{'qa_pairs': {'query': '这款全自动咖啡机有哪些尺寸选项？',
   'answer': "这款全自动咖啡机有两种尺寸选项：大型 - 尺寸为13.8'' x 17.3''；中型 - 尺寸为11.5'' x 15.2''。"}},
 {'qa_pairs': {'query': '根据文档中的描述，这款电动牙刷有哪些清洁模式和功能？',
   'answer': '文档中提到，这款电动牙刷具有多种清洁模式和其他特性，具体包括定时功能。'}},
 {'qa_pairs': {'query': '这款维生素C泡腾片每盒含有多少片？', 'answer': '每盒含有20片。'}},
 {'qa_pairs': {'query': '这款无线蓝牙耳机有哪些主要特点和规格？',
   'answer': '这款无线蓝牙耳机的主要特点包括：配备了降噪技术、长达8小时的电池续航力、快速充电功能以及内置麦克风支持接听电话。其规格方面，单个耳机尺寸为1.5英寸 x 1.3英寸，并且由耐用的塑料和金属构成，配备有软质耳塞。此外，清洁时只需用湿布擦拭即可。这款产品是在韩国制造的。'}},
 {'qa_pairs': {'query': '这款瑜伽垫的尺寸是多少？', 'answer': '这款瑜伽垫的尺寸是24英寸 x 68英寸。'}}]